In [2]:
import pandas as pd
import torch

In [3]:
splits = {'train': 'train.csv', 'test': 'test.csv'}
train_df = pd.read_csv("hf://datasets/osyvokon/pavlick-formality-scores/" + splits["train"])
test_df = pd.read_csv("hf://datasets/osyvokon/pavlick-formality-scores/" + splits["test"])
df = pd.concat([train_df, test_df])


In [4]:
# Load model directly
from transformers import AutoTokenizer, AutoModelForSequenceClassification

tokenizer = AutoTokenizer.from_pretrained("s-nlp/xlmr_formality_classifier")
model = AutoModelForSequenceClassification.from_pretrained("s-nlp/xlmr_formality_classifier")

C:\Users\ferga\AppData\Roaming\Python\Python311\site-packages\huggingface_hub\file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


: 

In [ ]:
id2formality = {0: "formal", 1: "informal"}

# Process texts in smaller batches to reduce memory usage
batch_size = 16  # Adjust batch size as needed
formality_scores = []

for i in range(0, len(df), batch_size):
    texts = list(df['sentence'][i:i + batch_size])
    print(texts)

    # prepare the input
    encoding = tokenizer(
        texts,
        add_special_tokens=True,
        return_token_type_ids=True,
        truncation=True,
        padding="max_length",
        return_tensors="pt",
    )

    # inference
    output = model(**encoding)

    # Collect formality scores for the current batch
    batch_scores = [
        {id2formality[idx]: score for idx, score in enumerate(text_scores.tolist())}
        for text_scores in output.logits.softmax(dim=1)
    ]
    formality_scores.extend(batch_scores)

print(formality_scores)

['Tang was employed at private-equity firm Friedman Fleischer & Lowe.', "San Francisco Mayor Gavin Newsom's withdrawal from the governor's race followed a meeting with top advisers where he was told that, unless he raised $5 million quickly and appeared to be viable, some of the state's biggest unions would throw their support behind Attorney General Jerry Brown.", 'lol nothing worrying about that.', 'She told Price she wanted to join the Police Explorers, a Boy Scouts group that lets boys and girls learn law enforcement with local police.', 'The prime minister is keen to use the autumn pre-budget statement to announce a new "fiscal stimulus", with billions of pounds of extra money for housing, infrastructure projects and training.', "Those competencies include mastering fundamental legal skills, support of the firm's culture, demonstration of leadership and business skills, and understanding and effectively managing client needs.", 'His platform contains plans to fund drainage project